# Social Network Link Analysis (Tutorial Notebook)

This notebook creates a **sample social network** and performs **link analysis** for a Recommendation Systems tutorial.

## What you will compute

### Node-level measures
- Degree (in / out), **degree centrality**
- **Closeness centrality** (directed & undirected), **harmonic centrality**
- **Betweenness centrality** (unweighted & weighted)
- **Eigenvector centrality** (with safe fallback) and **PageRank**
- **Local clustering coefficient**

### Network-level measures
- **Diameter**, **average geodesic distance**
- **Average degree**, **density**
- **Reciprocity**
- Components (weak/strong), **transitivity**, triangles, assortativity

### Bonus (useful for friend recommendation)
- Link prediction scores: **Adamic-Adar**, **Jaccard**, **Preferential Attachment**


In [ ]:
# Install (if needed):
# !pip install networkx matplotlib pandas numpy

import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 120)


: 

## 1) Create a sample social network (directed + weighted)

**Interpretation:** an edge `u -> v` means **u follows v** (or u interacts with v).  
Edge **weight** represents interaction strength (likes/messages/clicks/etc.).


In [ ]:
# -----------------------------
# Build a toy directed network
# -----------------------------
G = nx.DiGraph()

# Add nodes (people/users)
users = ["Asha", "Bimal", "Chamara", "Dinu", "Eshan", "Fathima", "Gayan", "Hasini", "Iresha", "Janith", "Kasun", "Lahiru"]
G.add_nodes_from(users)

# Add directed, weighted edges (u follows v) with weight = interaction strength
edges = [
    ("Asha", "Bimal", 3), ("Asha", "Chamara", 1), ("Asha", "Hasini", 2),
    ("Bimal", "Asha", 2), ("Bimal", "Chamara", 4), ("Bimal", "Dinu", 1),
    ("Chamara", "Bimal", 2), ("Chamara", "Eshan", 3), ("Chamara", "Fathima", 1),
    ("Dinu", "Chamara", 2), ("Dinu", "Eshan", 1),
    ("Eshan", "Fathima", 5), ("Eshan", "Gayan", 2), ("Eshan", "Hasini", 1),
    ("Fathima", "Eshan", 2), ("Fathima", "Hasini", 3),
    ("Gayan", "Eshan", 1), ("Gayan", "Hasini", 2), ("Gayan", "Iresha", 1),
    ("Hasini", "Asha", 1), ("Hasini", "Iresha", 4), ("Hasini", "Janith", 2),
    ("Iresha", "Hasini", 2), ("Iresha", "Janith", 2), ("Iresha", "Kasun", 1),
    ("Janith", "Kasun", 5), ("Janith", "Lahiru", 2),
    ("Kasun", "Janith", 1), ("Kasun", "Lahiru", 3),
    ("Lahiru", "Kasun", 2),

    # a couple of “bridges” across groups
    ("Dinu", "Hasini", 1),
    ("Kasun", "Eshan", 1),
]

G.add_weighted_edges_from(edges, weight="weight")

print("Nodes:", G.number_of_nodes())
print("Edges:", G.number_of_edges())


## 2) Quick visualization (optional)

We use a spring layout. Edge width is proportional to interaction weight.


In [ ]:
plt.figure(figsize=(10, 7))
pos = nx.spring_layout(G, seed=42)  # stable layout
nx.draw_networkx_nodes(G, pos, node_size=700)
nx.draw_networkx_labels(G, pos, font_size=9)

# edge width based on weight (scaled)
weights = np.array([G[u][v]["weight"] for u, v in G.edges()])
widths = 0.5 + (weights / weights.max()) * 3

nx.draw_networkx_edges(G, pos, arrows=True, width=widths, alpha=0.6)
plt.title("Sample Social Network (Directed, Weighted)")
plt.axis("off")
plt.show()


## 3) Helper functions for ranked outputs

These make it easy to show “Top-k nodes” under each metric.


In [ ]:
def to_ranked_df(metric_dict, top_n=None, ascending=False, col_name="score"):
    """Convert a dictionary {node: value} to a sorted DataFrame."""
    df = pd.DataFrame(metric_dict.items(), columns=["node", col_name])
    df = df.sort_values(col_name, ascending=ascending).reset_index(drop=True)
    if top_n:
        df = df.head(top_n)
    return df

def show_top(title, metric_dict, col_name, k=10):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))
    display(to_ranked_df(metric_dict, col_name=col_name).head(k))


## 4) Node-level measures (centralities + clustering)

### 4.1 Degree & Degree Centrality
- **In-degree**: number of followers (popularity)
- **Out-degree**: number of people you follow (activity)
- **Degree centrality**: normalized degree in [0,1]


In [ ]:
in_deg = dict(G.in_degree())
out_deg = dict(G.out_degree())
total_deg = dict(G.degree())

degree_centrality = nx.degree_centrality(G)

show_top("Top by In-Degree (followers)", in_deg, "in_degree", k=12)
show_top("Top by Out-Degree (following)", out_deg, "out_degree", k=12)
show_top("Top by Degree Centrality", degree_centrality, "degree_centrality", k=12)


### 4.2 Closeness Centrality (and Harmonic Centrality)

**Closeness** measures how close a node is to all others via shortest paths.

⚠️ In a **directed graph**, some nodes may not reach others, which changes closeness values.
Common tutorial practice:
- compute closeness on the **undirected** version, OR
- use **harmonic centrality** (robust when disconnected).

We compute all three:
- Directed closeness
- Undirected closeness
- Harmonic centrality


In [ ]:
closeness_directed = nx.closeness_centrality(G)

G_und = G.to_undirected()
closeness_undirected = nx.closeness_centrality(G_und)

harmonic = nx.harmonic_centrality(G)  # robust alternative

show_top("Top by Closeness (directed)", closeness_directed, "closeness_dir", k=12)
show_top("Top by Closeness (undirected)", closeness_undirected, "closeness_und", k=12)
show_top("Top by Harmonic Centrality (robust)", harmonic, "harmonic", k=12)


### 4.3 Betweenness Centrality

**Betweenness** captures “broker” nodes that lie on many shortest paths.

We compute:
- Unweighted betweenness (all edges equal distance)
- Weighted betweenness where **distance = 1 / weight** (stronger ties → shorter distance)


In [ ]:
betweenness = nx.betweenness_centrality(G, normalized=True)

# For weighted shortest paths: treat strength as inverse distance
G_dist = G.copy()
for u, v, d in G_dist.edges(data=True):
    d["distance"] = 1 / d["weight"]

betweenness_weighted = nx.betweenness_centrality(G_dist, normalized=True, weight="distance")

show_top("Top by Betweenness (unweighted)", betweenness, "betweenness", k=12)
show_top("Top by Betweenness (weighted via 1/weight distance)", betweenness_weighted, "betweenness_w", k=12)


### 4.4 Eigenvector Centrality and PageRank (Link Analysis)

- **Eigenvector centrality**: important if connected to important nodes.
- **PageRank**: classic link analysis for directed graphs; often more stable in practice.

If eigenvector centrality fails to converge, use PageRank.


In [ ]:
try:
    eig = nx.eigenvector_centrality(G, max_iter=2000, tol=1e-8)
    show_top("Top by Eigenvector Centrality", eig, "eigenvector", k=12)
except nx.PowerIterationFailedConvergence:
    eig = None
    print("Eigenvector centrality did not converge for this graph. Using PageRank is recommended.")

pagerank = nx.pagerank(G, alpha=0.85, weight="weight")
show_top("Top by PageRank (weighted)", pagerank, "pagerank", k=12)


### 4.5 Local Clustering Coefficient

For directed graphs, clustering is more complex.
A common social-network tutorial approach is to compute clustering on the **undirected** version.

Interpretation: “friend-of-friend” triangle density around a node.


In [ ]:
local_clust = nx.clustering(G_und)
show_top("Top by Local Clustering Coefficient", local_clust, "local_clustering", k=12)


## 5) Combined node-level results table

Useful for teaching: compare multiple metrics side-by-side.


In [ ]:
df_nodes = pd.DataFrame({"node": list(G.nodes())})

df_nodes["in_degree"] = df_nodes["node"].map(in_deg)
df_nodes["out_degree"] = df_nodes["node"].map(out_deg)
df_nodes["degree_centrality"] = df_nodes["node"].map(degree_centrality)
df_nodes["closeness_undirected"] = df_nodes["node"].map(closeness_undirected)
df_nodes["harmonic"] = df_nodes["node"].map(harmonic)
df_nodes["betweenness"] = df_nodes["node"].map(betweenness)
df_nodes["pagerank"] = df_nodes["node"].map(pagerank)
df_nodes["local_clustering"] = df_nodes["node"].map(local_clust)

if eig is not None:
    df_nodes["eigenvector"] = df_nodes["node"].map(eig)

df_nodes = df_nodes.sort_values("pagerank", ascending=False).reset_index(drop=True)
df_nodes


## 6) Network-level statistical measures

### Notes (diameter & average distance)
Distance measures require a connected graph. For directed graphs:
- We take the **largest weakly connected component** (WCC)
- Convert it to **undirected** for geodesic calculations

This is a standard tutorial-friendly approach.


In [ ]:
n = G.number_of_nodes()
m = G.number_of_edges()

density = nx.density(G)
reciprocity = nx.reciprocity(G)

avg_in_degree = np.mean(list(in_deg.values()))
avg_out_degree = np.mean(list(out_deg.values()))
avg_total_degree = np.mean(list(total_deg.values()))

weak_components = list(nx.weakly_connected_components(G))
strong_components = list(nx.strongly_connected_components(G))

largest_weak = max(weak_components, key=len)
largest_strong = max(strong_components, key=len)

G_wcc = G.subgraph(largest_weak).copy()
G_wcc_und = G_wcc.to_undirected()

diameter = nx.diameter(G_wcc_und)
avg_geodesic = nx.average_shortest_path_length(G_wcc_und)

transitivity = nx.transitivity(G_wcc_und)
triangles = sum(nx.triangles(G_wcc_und).values()) // 3

try:
    assortativity = nx.degree_assortativity_coefficient(G_wcc_und)
except Exception:
    assortativity = np.nan

network_stats = {
    "num_nodes": n,
    "num_edges": m,
    "density": density,
    "reciprocity": reciprocity,
    "avg_in_degree": avg_in_degree,
    "avg_out_degree": avg_out_degree,
    "avg_total_degree": avg_total_degree,
    "num_weak_components": len(weak_components),
    "num_strong_components": len(strong_components),
    "largest_weak_component_size": len(largest_weak),
    "largest_strong_component_size": len(largest_strong),
    "diameter_on_largest_WCC_undirected": diameter,
    "avg_geodesic_on_largest_WCC_undirected": avg_geodesic,
    "transitivity_on_largest_WCC_undirected": transitivity,
    "num_triangles_on_largest_WCC_undirected": triangles,
    "degree_assortativity_on_largest_WCC_undirected": assortativity,
}

pd.DataFrame(network_stats.items(), columns=["measure", "value"])


## 7) Bonus: Link prediction signals (for “Who to follow?” recommendations)

Common link prediction scores (often computed on undirected graphs):
- **Adamic-Adar**: rewards shared neighbors that are not “too popular”
- **Jaccard**: common_neighbors / union_neighbors
- **Preferential Attachment**: degree(u) * degree(v) (rich-get-richer)

We rank non-connected pairs as candidate recommendations.


In [ ]:
UG = G.to_undirected()
non_edges = list(nx.non_edges(UG))

# Adamic-Adar
aa_scores = list(nx.adamic_adar_index(UG, non_edges))
df_aa = pd.DataFrame(aa_scores, columns=["u", "v", "adamic_adar"]).sort_values("adamic_adar", ascending=False).head(15)
df_aa.reset_index(drop=True)


In [ ]:
# Jaccard coefficient
jc_scores = list(nx.jaccard_coefficient(UG, non_edges))
df_jc = pd.DataFrame(jc_scores, columns=["u", "v", "jaccard"]).sort_values("jaccard", ascending=False).head(15)

# Preferential attachment
pa_scores = list(nx.preferential_attachment(UG, non_edges))
df_pa = pd.DataFrame(pa_scores, columns=["u", "v", "pref_attach"]).sort_values("pref_attach", ascending=False).head(15)

display(df_jc.reset_index(drop=True))
display(df_pa.reset_index(drop=True))


## 8) Short concept notes (for your slides)

- **In-degree / Out-degree**: popularity vs activity in directed networks  
- **Degree centrality**: normalized connectivity  
- **Closeness**: how quickly a node can reach others (short paths)  
- **Harmonic**: robust closeness-like measure for disconnected graphs  
- **Betweenness**: brokers/bridges between communities  
- **Eigenvector**: importance via connections to important nodes  
- **PageRank**: stable link-analysis importance for directed graphs  
- **Local clustering**: triangle density around a node (community tightness)  
- **Reciprocity**: how mutual relationships are  
- **Diameter / Avg distance**: small-world indicators (computed on connected component)
